In [1]:
# https://towardsdatascience.com/face-detection-using-mtcnn-a-guide-for-face-extraction-with-a-focus-on-speed-c6d59f82d49
# https://www.kaggle.com/code/timesler/fast-mtcnn-detector-55-fps-at-full-resolution/notebook
# https://www.kaggle.com/code/timesler/guide-to-mtcnn-in-facenet-pytorch/notebook

In [2]:
# !pip install facenet-pytorch # install if not

In [3]:
# ref detector: https://towardsdatascience.com/face-detection-using-mtcnn-a-guide-for-face-extraction-with-a-focus-on-speed-c6d59f82d49
# ref rotation: https://www.kaggle.com/code/gpiosenka/align-crop-resize-save-images-using-mtcnn

import os
import sys
import cv2
import json
import numpy as np
from tqdm import tqdm
from utils import load_image
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN
from imutils.paths import list_images
from scipy.spatial.distance import euclidean

/home/choppy/TOC/.conda/envs/msc-thesis-2022/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
detector = MTCNN(select_largest=False, post_process=False, device='cuda:0', margin=40)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
datasets = {
    # 'flickr': '/media/choppy/WD_BLACK/datasets/FLICKR',
    # 'splunk': '/media/choppy/WD_BLACK/datasets/Splunk',
    
    'flickr': '/home/choppy/TOC/thesis-msc-2022/data/01_raw/FLICKR',
}

verbose = True

In [ ]:
i = 0

for dst_key in datasets:
    ds_dir = datasets.get(dst_key)
    # images = list(sorted([*list_images(ds_dir)], reverse=True))
    images = [*list_images(ds_dir)]
    # np.random.shuffle(images)
    db_name = os.path.basename(ds_dir)
    json_data = []
    
    print(f"doing inference over {len(images)} images from {db_name}")
    
    for imdir in tqdm(images, desc=dst_key):
        img = load_image(imdir)
        img = cv2.resize(img, (150, 150))
        
        if img is None:
            if verbose: print(f'img {imdir} could not be loaded. Check')
            continue
        detections = detector.detect(img, landmarks=True)
        
        
        if i == 5: break
        i += 1
        
    break

In [ ]:
l = 0

img = load_image(np.random.choice(images))
img = cv2.resize(img, (250, 250))
bboxes, confs, lands = detector.detect(img, landmarks=True)

nose_pt1, nose_pt2 = lands[l][0]
eyeizq_pt1, eyeizq_pt2 = lands[l][1]
eyeder_pt1, eyeder_pt2 = lands[l][2]
_pt1, _pt2 = lands[l][3]
__pt1, __pt2 = lands[l][4]

radii = 5

img = cv2.circle(img, (int(nose_pt1), int(nose_pt2)), radii, color=[255, 0, 0], thickness=-1)
img = cv2.circle(img, (int(eyeizq_pt1), int(eyeizq_pt2)), radii, color=[0, 255, 0], thickness=-1)
img = cv2.circle(img, (int(eyeder_pt1), int(eyeder_pt2)), radii, color=[0, 0, 255], thickness=-1)
img = cv2.circle(img, (int(_pt1), int(_pt2)), radii, color=[0, 0, 255], thickness=-1)
img = cv2.circle(img, (int(__pt1), int(__pt2)), radii, color=[0, 0, 255], thickness=-1)

plt.imshow(img)

In [ ]:
# nota: el mtcnn de pytorch funciona igual lentito, pero al menos no se come toda la memoria de gpu